![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/b8be43cf89c540bfaf3831f2c8506614).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

## Transformación información en Train

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


#### Anaálisis variable Model:
Evidenciamos que la variable 'Model' puede tomar 525 valores diferentes, y muchos de ellos no representan más del 0,96% de las observaciones totales, por lo que nos permitimos renombrar dichos valores como "Otros", dado que los valores restantes (Silverado, Grand, Sierra, Accord, F-1504WD, Civic, Wrangler, 3, Santa, Jetta, FusionSE, EquinoxFWD, CamrySE y Super) representan más del 22% de las observaciones totales.

In [ ]:
print(dataTraining['Model'].unique())
print(dataTraining['Model'].nunique())

In [ ]:
valores_unicos = dataTraining['Model'].unique()
for valor in valores_unicos:
    porcentaje = (dataTraining['Model'] == valor).mean() * 100
    print(f"El {porcentaje:.2f}% de las observaciones tienen el valor '{valor}' en la columna 'Model'")

In [ ]:
# Calculamos el porcentaje de observaciones para cada valor único en la columna 'Model'
porcentajes = dataTraining['Model'].value_counts(normalize=True) * 100

# Renombra,os los valores que no representen más del 0.96% de las observaciones como 'Otro'
valores_permitidos = porcentajes[porcentajes > 0.96].index
dataTraining['Model'] = dataTraining['Model'].where(dataTraining['Model'].isin(valores_permitidos), 'Otro')

#### Análisis variable Make:
Similar a lo acontecido con la variabnle 'Model', evidenciamos que la variable 'Make' puede tomar 38 valores diferentes, y muchos de ellos no representan más del 2,0% de las observaciones totales, por lo que nos permitimos renombrar dichos valores como "Otro", dado que los valores restantes representan más del 87% de las observaciones totales.

In [ ]:
print(dataTraining['Make'].unique())
print(dataTraining['Make'].nunique())

In [ ]:
valores_unicos = dataTraining['Make'].unique()
for valor in valores_unicos:
    porcentaje = (dataTraining['Make'] == valor).mean() * 100
    print(f"El {porcentaje:.2f}% de las observaciones tienen el valor '{valor}' en la columna 'Make'")

In [ ]:
# Calculamos el porcentaje de observaciones para cada valor único en la columna 'Model'
porcentajes = dataTraining['Make'].value_counts(normalize=True) * 100

# Renombra,os los valores que no representen más del 2% de las observaciones como 'Otro'
valores_permitidos = porcentajes[porcentajes > 2].index
dataTraining['Make'] = dataTraining['Make'].where(dataTraining['Make'].isin(valores_permitidos), 'Otro')

In [5]:
#Factorizacion variables categoricas
dataTraining['State'] = pd.factorize(dataTraining.State)[0]
dataTraining['Make'] = pd.factorize(dataTraining.Make)[0]
dataTraining['Model'] = pd.factorize(dataTraining.Model)[0]
dataTraining['Year'] = pd.factorize(dataTraining.Year)[0]

### Transformación variables en dummies:

In [5]:
# convertimos la columna 'Make' en variables dummy
dummies = pd.get_dummies(dataTraining['Make'], prefix='Make')
dataTraining = pd.concat([dataTraining, dummies], axis=1)
dataTraining = dataTraining.drop('Make', axis=1)
# Muestra el resultado
print(dataTraining)

        Price  Year  Mileage State           Model  Make_Acura  Make_Audi  \
0       34995  2017     9913    FL        Wrangler           0          0   
1       37895  2015    20578    OH        Tahoe4WD           0          0   
2       18430  2012    83716    TX           X5AWD           0          0   
3       24681  2014    28729    OH       SRXLuxury           0          0   
4       26998  2013    64032    CO        Wrangler           0          0   
...       ...   ...      ...   ...             ...         ...        ...   
399995  29900  2015    25287    TX            RXRX           0          0   
399996  17688  2015    17677    MI      EquinoxFWD           0          0   
399997  24907  2014    66688    NC  EnclaveLeather           0          0   
399998  11498  2014    37872    IN           Jetta           0          0   
399999  16900  2014    78606    CO     PathfinderS           0          0   

        Make_BMW  Make_Bentley  Make_Buick  ...  Make_Pontiac  Make_Porsche

In [6]:
# convertimos la columna 'Model' en variables dummy
dummies = pd.get_dummies(dataTraining['Model'], prefix='Model')
dataTraining = pd.concat([dataTraining, dummies], axis=1)
dataTraining = dataTraining.drop('Model', axis=1)
# Muestra el resultado
print(dataTraining)

        Price  Year  Mileage State  Make_Acura  Make_Audi  Make_BMW  \
0       34995  2017     9913    FL           0          0         0   
1       37895  2015    20578    OH           0          0         0   
2       18430  2012    83716    TX           0          0         1   
3       24681  2014    28729    OH           0          0         0   
4       26998  2013    64032    CO           0          0         0   
...       ...   ...      ...   ...         ...        ...       ...   
399995  29900  2015    25287    TX           0          0         0   
399996  17688  2015    17677    MI           0          0         0   
399997  24907  2014    66688    NC           0          0         0   
399998  11498  2014    37872    IN           0          0         0   
399999  16900  2014    78606    CO           0          0         0   

        Make_Bentley  Make_Buick  Make_Cadillac  ...  Model_Yaris4dr  \
0                  0           0              0  ...               0   
1  

In [7]:
# convertimos la columna 'Year' en variables dummy
dataTraining['Year'] = pd.factorize(dataTraining.Year)[0]
# Muestra el resultado
print(dataTraining)

        Price  Year  Mileage State  Make_Acura  Make_Audi  Make_BMW  \
0       34995     0     9913    FL           0          0         0   
1       37895     1    20578    OH           0          0         0   
2       18430     2    83716    TX           0          0         1   
3       24681     3    28729    OH           0          0         0   
4       26998     4    64032    CO           0          0         0   
...       ...   ...      ...   ...         ...        ...       ...   
399995  29900     1    25287    TX           0          0         0   
399996  17688     1    17677    MI           0          0         0   
399997  24907     3    66688    NC           0          0         0   
399998  11498     3    37872    IN           0          0         0   
399999  16900     3    78606    CO           0          0         0   

        Make_Bentley  Make_Buick  Make_Cadillac  ...  Model_Yaris4dr  \
0                  0           0              0  ...               0   
1  

In [8]:
# convertimos la columna 'State' en variables dummy
dummies = pd.get_dummies(dataTraining['State'], prefix='State')
dataTraining = pd.concat([dataTraining, dummies], axis=1)
dataTraining = dataTraining.drop('State', axis=1)
# Muestra el resultado
print(dataTraining)

        Price  Year  Mileage  Make_Acura  Make_Audi  Make_BMW  Make_Bentley  \
0       34995     0     9913           0          0         0             0   
1       37895     1    20578           0          0         0             0   
2       18430     2    83716           0          0         1             0   
3       24681     3    28729           0          0         0             0   
4       26998     4    64032           0          0         0             0   
...       ...   ...      ...         ...        ...       ...           ...   
399995  29900     1    25287           0          0         0             0   
399996  17688     1    17677           0          0         0             0   
399997  24907     3    66688           0          0         0             0   
399998  11498     3    37872           0          0         0             0   
399999  16900     3    78606           0          0         0             0   

        Make_Buick  Make_Cadillac  Make_Chevrolet  

## Transformación información en Test

In [ ]:
# Visualización datos de test
dataTesting.head()

#### Análisis variable Model:
Realizamos el mismo procedimiento que con la variable en Train y todos las observaciones que no tomen los valores de los modelos más significativos (Silverado, Grand, Sierra, Accord, F-1504WD, Civic, Wrangler, 3, Santa, Jetta, FusionSE, EquinoxFWD, CamrySE y Super) las renombramos como 'Otro'.

In [ ]:
# Calculamos el porcentaje de observaciones para cada valor único en la columna 'Model'
porcentajes = dataTesting['Model'].value_counts(normalize=True) * 100

# Renombra,os los valores que no representen más del 1% de las observaciones como 'Otro'
valores_permitidos = porcentajes[porcentajes > 1].index
dataTesting['Model'] = dataTesting['Model'].where(dataTesting['Model'].isin(valores_permitidos), 'Otro')

#### Análisis variable Make:
Realizamos el mismo procedimiento que con la variable en Train y todos las observaciones que no tomen los valores de los modelos más significativos (>2% de las observaciones totales) las renombramos como 'Otro'.

In [ ]:
# Calculamos el porcentaje de observaciones para cada valor único en la columna 'Model'
porcentajes = dataTesting['Make'].value_counts(normalize=True) * 100

# Renombra,os los valores que no representen más del 2% de las observaciones como 'Otro'
valores_permitidos = porcentajes[porcentajes > 2].index
dataTesting['Make'] = dataTesting['Make'].where(dataTesting['Make'].isin(valores_permitidos), 'Otro')

In [6]:
dataTesting['State'] = pd.factorize(dataTesting.State)[0]
dataTesting['Make'] = pd.factorize(dataTesting.Make)[0]
dataTesting['Model'] = pd.factorize(dataTesting.Model)[0]
dataTesting['Year'] = pd.factorize(dataTesting.Year)[0]

### Transformación variables en dummies:

In [9]:
# convertimos la columna 'Make' en variables dummy
dummies = pd.get_dummies(dataTesting['Make'], prefix='Make')
dataTesting = pd.concat([dataTesting, dummies], axis=1)
dataTesting = dataTesting.drop('Make', axis=1)
# Muestra el resultado
print(dataTesting)

       Year  Mileage State         Model  Make_Acura  Make_Audi  Make_BMW  \
ID                                                                          
0      2014    31909    MD     MuranoAWD           0          0         0   
1      2017     5362    FL      Wrangler           0          0         0   
2      2014    50300    OH   FlexLimited           0          0         0   
3      2004   132160    WA             5           0          0         1   
4      2015    25226    MA         Grand           0          0         0   
...     ...      ...   ...           ...         ...        ...       ...   
99995  2015    82719    TX        Savana           0          0         0   
99996  2015    19711    UT             5           0          0         1   
99997  2016    48049    CA     MuranoAWD           0          0         0   
99998  2017    15306    CO  PatriotSport           0          0         0   
99999  2013    40393    TX             1           0          0         1   

In [10]:
# convertimos la columna 'Model' en variables dummy
dummies = pd.get_dummies(dataTesting['Model'], prefix='Model')
dataTesting = pd.concat([dataTesting, dummies], axis=1)
dataTesting = dataTesting.drop('Model', axis=1)
# Muestra el resultado
print(dataTesting)

       Year  Mileage State  Make_Acura  Make_Audi  Make_BMW  Make_Bentley  \
ID                                                                          
0      2014    31909    MD           0          0         0             0   
1      2017     5362    FL           0          0         0             0   
2      2014    50300    OH           0          0         0             0   
3      2004   132160    WA           0          0         1             0   
4      2015    25226    MA           0          0         0             0   
...     ...      ...   ...         ...        ...       ...           ...   
99995  2015    82719    TX           0          0         0             0   
99996  2015    19711    UT           0          0         1             0   
99997  2016    48049    CA           0          0         0             0   
99998  2017    15306    CO           0          0         0             0   
99999  2013    40393    TX           0          0         1             0   

In [11]:
# convertimos la columna 'Year' en variables dummy
dataTesting['Year'] = pd.factorize(dataTesting.Year)[0]
# Muestra el resultado
print(dataTesting)

       Year  Mileage State  Make_Acura  Make_Audi  Make_BMW  Make_Bentley  \
ID                                                                          
0         0    31909    MD           0          0         0             0   
1         1     5362    FL           0          0         0             0   
2         0    50300    OH           0          0         0             0   
3         2   132160    WA           0          0         1             0   
4         3    25226    MA           0          0         0             0   
...     ...      ...   ...         ...        ...       ...           ...   
99995     3    82719    TX           0          0         0             0   
99996     3    19711    UT           0          0         1             0   
99997     4    48049    CA           0          0         0             0   
99998     1    15306    CO           0          0         0             0   
99999     5    40393    TX           0          0         1             0   

In [12]:
# convertimos la columna 'State' en variables dummy
dummies = pd.get_dummies(dataTesting['State'], prefix='State')
dataTesting = pd.concat([dataTesting, dummies], axis=1)
dataTesting = dataTesting.drop('State', axis=1)
# Muestra el resultado
print(dataTesting)

       Year  Mileage  Make_Acura  Make_Audi  Make_BMW  Make_Bentley  \
ID                                                                    
0         0    31909           0          0         0             0   
1         1     5362           0          0         0             0   
2         0    50300           0          0         0             0   
3         2   132160           0          0         1             0   
4         3    25226           0          0         0             0   
...     ...      ...         ...        ...       ...           ...   
99995     3    82719           0          0         0             0   
99996     3    19711           0          0         1             0   
99997     4    48049           0          0         0             0   
99998     1    15306           0          0         0             0   
99999     5    40393           0          0         1             0   

       Make_Buick  Make_Cadillac  Make_Chevrolet  Make_Chrysler  ...  \
ID  

#### Definimos X_train, y_train y x_test

In [ ]:
#Particion de x-y datos training y test
#X_train = dataTraining.drop(["Price"], axis = 1)  
#y_train = dataTraining.filter(["Price"], axis = 1)
#X_test = dataTesting

In [7]:
#Particion de x-y datos training
XTotal = dataTraining.drop(["Price"], axis = 1)  
yTotal = dataTraining.filter(["Price"], axis = 1)

# particion de training para calculo de MSE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XTotal, yTotal, test_size=0.20, random_state=0)

### Bagging

### Random Forest

### Adaboost

### Gradient Boosting

### XGBoost

In [8]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
clf = XGBRegressor()
clf

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [9]:
from sklearn import metrics
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [10]:
# Calcula el MSE usando validación cruzada
#scores = cross_val_score(clf, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
#mse = np.mean(-scores)

#print(f"MSE estimado: {mse:.2f}")

In [10]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

MSE: 15424188.753508365


In [11]:
rmse = sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 3927.364097395143


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

#### Calibramos parámetros de XGBoost

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Definimos los valores a probar para cada parámetro
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'gamma': [0, 1, 5],
    'colsample_bytree': [0.3, 0.7, 1]
}

In [ ]:
# Creamos un objeto XGBRegressor
modelo1 = XGBRegressor()

# Creamos un objeto GridSearchCV
grid_search = GridSearchCV(modelo1, param_grid, cv=5)

In [ ]:
# Realizamos la búsqueda en cuadrícula para encontrar los mejores parámetros utilizando validación cruzada
grid_search.fit(XTotal, yTotal)

In [ ]:
# Se muestran los mejores parámetros encontrados
print(grid_search.best_params_)

In [ ]:
clf = XGBRegressor(learning_rate=0.1,gamma=0,colsample_bytree=0.7)
clf

In [ ]:
# Entrenamiento (fit) y desempeño del modelo XGBRegressor
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
#Cálculo de MSE
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

In [ ]:
rmse = sqrt(mse)
print(f'RMSE: {rmse}')

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

In [ ]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()